In [ ]:
import os
import sys
%pylab

import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    %pylab
from src import data

from src import efficient_frontier
from src import plot
from src import portfolio
from src import capm
import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds

Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\toros\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['plot']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [ ]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 0
max_esg_score = 2000
df = pd.read_excel('../data/ESG_DATA_S&P500.xlsx')
dates = ['2021-06-01','2022-01-01']
start_year = dates[0]
end_year = dates[1]

Bounds = Bounds(-1,2) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.01


In [ ]:
esg_data = data.esg_score_weight(df,weights,min_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data) #Gives us price data and esg data to use in future functions
pct_returns = data.pct_returns_from_prices(prices) #Gives us the returns in pct for our stocks
esgdata

c:\Users\toros\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:40: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\toros\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:113: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 0.000% => Return: 1.951%  Sharpe Ratio = 11462.88
Max. Sharpe Ratio = 196331.87 => Return: 15.12%  Risk: 0.000%


In [ ]:
parameters = parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds, sharpe_type,start_year,end_year, Wanted_return, maximum_risk) #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters) #Gives us the weights of the individual stocks in our portfolio
portfolio_esg_score = portfolio.esg_score_of_portfolio(weights_of_each_portfolio,esgdata.head(1)) #gives us the ESG score of our portfolio
portfolio_allocations = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[1] #How much we allocate to our market portfolio compared to the risk-free asset
cmle_returns = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[0] #Return based on how much we allocate to the risk free asset.
#sp500 = data.data_for_beta(['spy'].append(weights_of_each_portfolio.columns()),dates)
#pct_returns_sp500 = data.pct_returns_from_prices(sp500)


In [ ]:
weights_of_each_portfolio


,AWK,ANSS,APTV,ADM,AMZN,ALLE,AKAM,APD,ACN,ABT,...,TYL,URI,VRSN,VRSK,V,WM,WEC,WST,WAB,WY
0,0.076577,0.05071,0.032153,0.001152,-0.129394,-0.125087,-0.033121,0.002413,0.20699,0.022052,...,-0.004803,-0.1089,-0.002797,0.160941,-0.020876,0.069785,-0.05177,0.151424,-0.038776,0.112554


In [ ]:
portfolio_allocations

[129854.62802241462]

In [ ]:
portfolio_esg_score

,ESG_score_of_portfolio
0,1255.641073


In [ ]:
cmle_returns

[18334.650280939408]